In [2]:
from typing import Dict, Optional, Any
import os
import pickle
import collections

import mdtraj as md
import numpy as np
import scipy.stats
import pyemma
import pandas as pd
import lovelyplots
import matplotlib.pyplot as plt
plt.style.use("ipynb")

import matplotlib as mpl
mpl.rcParams['axes.formatter.useoffset'] = False
mpl.rcParams['axes.formatter.limits'] = (-10000, 10000)  # Controls range before scientific notation is used

import utils as analysis_utils
import pyemma_helper

from jamun import utils

### Environment Variables

These are needed to load reference MD trajectories for comparison.

In [3]:
# results_dir = "/homefs/home/daigavaa/jamun/analysis_results"
results_dir = "/data/bucket/kleinhej/jamun-analysis/"
print(f"results_dir: {results_dir}")

results_dir: /data/bucket/kleinhej/jamun-analysis/


In [4]:
traj_name = "JAMUN"
ref_traj_name = "MDGenReference"

### Load All Trajectories

In [5]:
output_dir = os.path.join("plots", traj_name, f"ref={ref_traj_name}")
print(f"Plots will be saved to {output_dir}")

results_path = os.path.join(results_dir, traj_name, f"ref={ref_traj_name}")
print(f"Searching for results in {results_path}")

results = []
for results_file in sorted(os.listdir(results_path)):
    peptide, ext = os.path.splitext(results_file)
    if ext != ".pkl":
        continue

    with open(os.path.join(results_path, results_file), "rb") as f:
        all_results = pickle.load(f)

    results.append({
        "traj": traj_name,
        "ref_traj": ref_traj_name,
        "peptide": peptide,
        "results": all_results["results"],
        "args": all_results["args"],
    })

results_df = pd.DataFrame(results)
results_df

Plots will be saved to plots/JAMUN/ref=MDGenReference
Searching for results in /data/bucket/kleinhej/jamun-analysis/JAMUN/ref=MDGenReference


,traj,ref_traj,peptide,results,args
0,JAMUN,MDGenReference,RVQS,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'RVQS', 'trajectory': 'JAMUN', 're..."
1,JAMUN,MDGenReference,SDFS,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SDFS', 'trajectory': 'JAMUN', 're..."
2,JAMUN,MDGenReference,SFCH,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SFCH', 'trajectory': 'JAMUN', 're..."
3,JAMUN,MDGenReference,SNIN,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SNIN', 'trajectory': 'JAMUN', 're..."
4,JAMUN,MDGenReference,SNSF,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SNSF', 'trajectory': 'JAMUN', 're..."
5,JAMUN,MDGenReference,SPFH,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SPFH', 'trajectory': 'JAMUN', 're..."
6,JAMUN,MDGenReference,SPVN,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SPVN', 'trajectory': 'JAMUN', 're..."
7,JAMUN,MDGenReference,SSNN,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SSNN', 'trajectory': 'JAMUN', 're..."
8,JAMUN,MDGenReference,TLRK,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'TLRK', 'trajectory': 'JAMUN', 're..."
9,JAMUN,MDGenReference,VDRN,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'VDRN', 'trajectory': 'JAMUN', 're..."


In [6]:
# Pick some rows randomly.
sampled_results_df = results_df.sample(n=4, random_state=42)
sampled_results_df = sampled_results_df.reset_index(drop=True)
sampled_results_df

,traj,ref_traj,peptide,results,args
0,JAMUN,MDGenReference,RVQS,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'RVQS', 'trajectory': 'JAMUN', 're..."
1,JAMUN,MDGenReference,SDFS,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SDFS', 'trajectory': 'JAMUN', 're..."
2,JAMUN,MDGenReference,SPFH,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'SPFH', 'trajectory': 'JAMUN', 're..."
3,JAMUN,MDGenReference,YMQN,{'featurization': {'traj': {'feats': {'torsion...,"{'peptide': 'YMQN', 'trajectory': 'JAMUN', 're..."


### Ramachandran Plots

In [7]:
def plot_ramachandran_contour(results: Dict[str, Any], dihedral_index: int, ax: Optional[plt.Axes] = None) -> plt.Axes:
    """Plots the Ramachandran contour plot of a trajectory."""

    if ax is None:
        _, ax = plt.subplots(figsize=(10, 10))

    pmf, xedges, yedges = results["pmf"], results["xedges"], results["yedges"]
    im = ax.contourf(xedges[:-1], yedges[:-1], pmf[dihedral_index], cmap="viridis", levels=50)
    contour = ax.contour(xedges[:-1], yedges[:-1], pmf[dihedral_index], colors="white", linestyles="solid", levels=30, linewidths=0.25)

    ax.set_aspect("equal", adjustable="box")
    ax.set_xlabel("$\phi$")
    ax.set_ylabel("$\psi$")

    tick_eps = 0.1
    ticks = [-np.pi + tick_eps, -np.pi / 2, 0, np.pi / 2, np.pi - tick_eps]
    tick_labels = ["$-\pi$", "$-\pi/2$", "$0$", "$\pi/2$", "$\pi$"]
    ax.set_xticks(ticks, tick_labels)
    ax.set_yticks(ticks, tick_labels)

    return ax


def format_for_plot(peptide: str) -> str:
    """Formats the peptide name for plotting."""
    return "".join([utils.convert_to_one_letter_code(aa) for aa in peptide])

In [ ]:
n_dihedrals = 2
if n_dihedrals % 2 == 0:
    label_offset = 0.0
else:
    label_offset = 0.5

fig, axs = plt.subplots(len(sampled_results_df), 2 * n_dihedrals, figsize=(32, 4 * len(sampled_results_df)), squeeze=False)
for i, row in sampled_results_df.iterrows():
    peptide = row["peptide"]

    for j in range(n_dihedrals):
        plot_ramachandran_contour(row["results"]["PMFs"]["ref_traj"]["pmf_internal"], j, axs[i, j])
        plot_ramachandran_contour(row["results"]["PMFs"]["traj"]["pmf_internal"], j, axs[i, j + n_dihedrals])

    # Add labels.
    axs[0, n_dihedrals // 2].text(
        label_offset,
        1.1,
        "Reference",
        horizontalalignment="center",
        verticalalignment="center",
        transform=axs[0, n_dihedrals // 2].transAxes,
    )
    axs[0, n_dihedrals // 2 + n_dihedrals].text(
        label_offset,
        1.1,
        "JAMUN",
        horizontalalignment="center",
        verticalalignment="center",
        transform=axs[0, n_dihedrals // 2 + n_dihedrals].transAxes,
    )
    axs[i, -1].text(
        1.1,
        0.5,
        format_for_plot(peptide),
        rotation=90,
        verticalalignment="center",
        horizontalalignment="center",
        transform=axs[i, -1].transAxes,
    )

plt.tight_layout()
plt.show()
plt.savefig(os.path.join(output_dir, "ramachandran_contours.pdf"), dpi=300)


### Feature Histograms

In [ ]:
fig, axs = plt.subplots(nrows=len(sampled_results_df), ncols=2, figsize=(14, 4 * len(sampled_results_df)), squeeze=False)
for i, row in sampled_results_df.iterrows():
    peptide = row["peptide"]

    feats = row["results"]["featurization"]
    histograms = row["results"]["feature_histograms"]

    pyemma_helper.plot_feature_histograms(
        histograms["ref_traj"]["torsions"]["histograms"],
        histograms["ref_traj"]["torsions"]["edges"],
        feature_labels=feats["ref_traj"]["feats"]["torsions"].describe(),
        ax=axs[i, 0]
    )

    pyemma_helper.plot_feature_histograms(
        histograms["traj"]["torsions"]["histograms"],
        histograms["traj"]["torsions"]["edges"],    
        feature_labels=feats["traj"]["feats"]["torsions"].describe(),
        ax=axs[i, 1]
    )

    axs[i, -1].text(
        1.1,
        0.5,
        format_for_plot(peptide),
        rotation=90,
        verticalalignment="center",
        horizontalalignment="center",
        transform=axs[i, -1].transAxes,
    )

axs[0, 0].set_title("Reference")
axs[0, 1].set_title("JAMUN")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "feature_histograms.pdf"), dpi=300)

In [ ]:
fig, axs = plt.subplots(nrows=len(sampled_results_df), ncols=2, figsize=(14, 4 * len(sampled_results_df)), squeeze=False)
for i, row in sampled_results_df.iterrows():
    peptide = row["peptide"]

    feats = row["results"]["featurization"]
    histograms = row["results"]["feature_histograms"]

    pyemma_helper.plot_feature_histograms(
        histograms["ref_traj"]["distances"]["histograms"][:10],
        histograms["ref_traj"]["distances"]["edges"][:10],
        feature_labels=feats["ref_traj"]["feats"]["distances"].describe()[:10],
        ax=axs[i, 0]
    )

    pyemma_helper.plot_feature_histograms(
        histograms["traj"]["distances"]["histograms"][:10],
        histograms["traj"]["distances"]["edges"][:10],    
        feature_labels=feats["traj"]["feats"]["distances"].describe()[:10],
        ax=axs[i, 1]
    )

    axs[i, -1].text(
        1.1,
        0.5,
        format_for_plot(peptide),
        rotation=90,
        verticalalignment="center",
        horizontalalignment="center",
        transform=axs[i, -1].transAxes,
    )

axs[0, 0].set_title("Reference")
axs[0, 1].set_title("JAMUN")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "distance_histograms.pdf"), dpi=300)

In [ ]:
all_JSDs = collections.defaultdict(list)
for i, row in results_df.iterrows():
    peptide = row["peptide"]
    for feat in ["backbone_torsions", "sidechain_torsions", "all_torsions"]:
        all_JSDs[feat].append(row["results"]["JSD_stats"][feat])

for feat in all_JSDs:
    print(feat, "mean JSD:", np.mean(all_JSDs[feat]))

# Save as a text file.
with open(os.path.join(output_dir, "JSDs.txt"), "w") as f:
    for feat in all_JSDs:
        f.write(f"{feat} mean JSD: {np.mean(all_JSDs[feat])}\n")


### TICA Analysis

In [ ]:
fig, axs = plt.subplots(nrows=len(sampled_results_df), ncols=2, figsize=(12, 3 * len(sampled_results_df)), squeeze=False)
for i, row in sampled_results_df.iterrows():
    peptide = row["peptide"]
    results = row["results"]["TICA_stats"]["TICA-0,1 histograms"]

    # Plot free energy.
    ref_traj_tica = results["ref_traj"]
    pyemma_helper.plot_free_energy(*ref_traj_tica, cmap="plasma", ax=axs[i, 0])
    axs[i, 0].set_title("Reference")
    axs[i, 0].ticklabel_format(useOffset=False, style="plain")

    traj_tica = results["traj"]
    pyemma_helper.plot_free_energy(*traj_tica, cmap="plasma", ax=axs[i, 1])
    axs[i, 1].set_title("JAMUN")
    axs[i, 1].ticklabel_format(useOffset=False, style="plain")

    # Set the same limits for both plots.
    axs[i, 1].set_xlim(axs[i, 0].get_xlim())
    axs[i, 1].set_ylim(axs[i, 0].get_ylim())
    axs[i, -1].text(
        1.4,
        0.5,
        format_for_plot(peptide),
        rotation=90,
        verticalalignment="center",
        horizontalalignment="center",
        transform=axs[i, -1].transAxes,
    )

plt.suptitle("TICA-0,1 Projections", fontsize="x-large")
plt.tight_layout()
plt.savefig(os.path.join(output_dir, "tica_projections.pdf"), dpi=300)

In [ ]:

fig, axs = plt.subplots(nrows=len(sampled_results_df), ncols=1, figsize=(8, 2 * len(sampled_results_df)), squeeze=False)

for i, row in sampled_results_df.iterrows():
    peptide = row["peptide"]
    results = row["results"]["autocorrelation_stats"]

    ref_autocorr = results["ref_autocorr"]
    traj_autocorr = results["traj_autocorr"]
    axs[i, 0].plot(ref_autocorr, label="Reference")
    axs[i, 0].plot(traj_autocorr, label="JAMUN")
    axs[i, 0].set_xlabel("Lag")
    axs[i, 0].set_ylabel("Autocorrelation")
    axs[i, 0].text(
        0.5,
        1.1,
        format_for_plot(peptide),
        rotation=0,
        verticalalignment="center",
        horizontalalignment="center",
        transform=axs[i, 0].transAxes,
        fontsize="medium",
    )

# Place legend outside plot.
plt.suptitle(f"TICA-0 Autocorrelation")
plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
plt.ticklabel_format(useOffset=False)
plt.tight_layout()
plt.show()
plt.savefig(os.path.join(output_dir, "tica_autocorrelation.pdf"), dpi=300)

In [ ]:
for i, row in results_df.iterrows():    

    results = row["results"]["JSD_stats_against_time"]

    ref_traj_results = results["ref_traj"]
    ref_traj_results = {key: result["backbone_torsions"] for key, result in ref_traj_results.items()}
    
    ref_traj_steps = np.array(list(ref_traj_results.keys()))
    ref_traj_js_divs = np.array(list(ref_traj_results.values()))
    ref_traj_progress = ref_traj_steps / np.max(ref_traj_steps)
    
    traj_results = results["traj"]
    traj_results = {key: result["backbone_torsions"] for key, result in traj_results.items()}
    
    traj_steps = np.array(list(traj_results.keys()))
    traj_js_divs = np.array(list(traj_results.values()))
    traj_progress = traj_steps / np.max(traj_steps)
    
    if i == len(sampled_results_df) - 1:
        ref_label = "Reference"
        traj_label = "JAMUN"
    else:
        ref_label = None
        traj_label = None
    
    plt.plot(ref_traj_progress, ref_traj_js_divs, color="C0", label=ref_label)
    plt.plot(traj_progress, traj_js_divs, color="C1", label=traj_label)

plt.title("Jenson-Shannon Distances on Backbone Torsions")
plt.xlabel("Fraction of Trajectory Progress", fontsize=12)
plt.ylabel("Jenson-Shannon Distance", fontsize=12)
plt.ticklabel_format(useOffset=False, style="plain")
plt.legend(fontsize=10)
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.show()
plt.savefig(os.path.join(output_dir, "jsd_against_time.pdf"), dpi=300)

### MSM Analysis

In [ ]:
all_ref_metastable_probs = []
all_traj_metastable_probs = []

for i, row in results_df.iterrows():    
    results = row["results"]["MSM_stats"]
    ref_metastable_probs = results["ref_metastable_probs"]
    traj_metastable_probs = results["traj_metastable_probs"]
    
    all_ref_metastable_probs.append(ref_metastable_probs)
    all_traj_metastable_probs.append(traj_metastable_probs)

all_ref_metastable_probs = np.concatenate(all_ref_metastable_probs)
all_traj_metastable_probs = np.concatenate(all_traj_metastable_probs)

plt.scatter(all_ref_metastable_probs, all_traj_metastable_probs)

# Fit line
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(all_ref_metastable_probs, all_traj_metastable_probs)

# Create x points for line (just use min and max)
x_line = np.array([-0.5, 1.5])
y_line = slope * x_line + intercept

# Plot the fitted line with dashed style
plt.plot(x_line, y_line, color='red', linestyle='--')

# Plot the fitted line
# plt.plot(all_ref_metastable_probs, line, color='gray', linestyle='--')
plt.text(0.60, 0.90, f'R² = {r_value**2:.3f}', transform=plt.gca().transAxes, color='red')

plt.title("Metastable State Probabilities")
plt.xlim((0, 1))
plt.ylim((0, 1))
plt.xlabel("Reference")
plt.ylabel("JAMUN")
plt.tight_layout()
plt.show()
plt.savefig(os.path.join(output_dir, "metastable_probs.pdf"), dpi=300)

In [ ]:
JSD_msms = []
for i, row in results_df.iterrows():    
    results = row["results"]["MSM_stats"]    
    JSD_msms.append(results["JSD_metastable_probs"])

JSD_msms = np.array(JSD_msms)
print("Mean JSD MSM:", np.mean(JSD_msms))

# Save as a text file.
with open(os.path.join(output_dir, "jsd_metastable_probs.txt"), "w") as f:
    f.write(f"Mean JSD MSM: {np.mean(JSD_msms)}\n")

plt.hist(JSD_msms)
plt.title("Metastable State Probabilities")
plt.xlabel("Jenson-Shannon Distance")
plt.ylabel("Frequency")
plt.ticklabel_format(useOffset=False, style="plain")
plt.tight_layout()
plt.show()
plt.savefig(os.path.join(output_dir, "jsd_metastable_probs.pdf"), dpi=300)